In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from datetime import datetime

from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import plot_partial_dependence

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Normalization
from tensorflow.keras import regularizers
import tensorflow.keras as keras
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers

import plotly.express as px

from utils import run_all_regressors
%load_ext autoreload
%autoreload 2

In [2]:
pd.set_option('display.max_columns', None)

Data Pre-processing and EDA(Exploratory data analysis)
Data Cleaning and filtering data which has firesize <5000 as number of small fires are high.

In [3]:
def akReadDf():
    # Reading the combined CSV files
    df = pd.read_csv('Wildfire.csv')
    df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1','disc_date_final','cont_date_final','cont_clean_date','putout_time'])
    df['disc_clean_date'] = pd.to_datetime(df['disc_clean_date'], format='%m/%d/%Y')

    #Get rid of outliers - fires of size larger than 5000 acres, and there are large number of small fires and other very less number are having the high 
    # area of fires, because of which the deviation is very high
    df = df.loc[df['fire_size'] < 5000]
    df.columns

    ################
    df['Vegetation'] = df['Vegetation'].astype('category')
    df['Cause'] = df['stat_cause_descr'].astype('category')

    df = pd.get_dummies(df,prefix=['Vegetation'], columns = ['Vegetation'], drop_first=True)
    df = pd.get_dummies(df,prefix=['Cause'], columns = ['stat_cause_descr'], drop_first=True)

    ################
    df_numerics_only = df.select_dtypes(include=np.number)

    corr = df_numerics_only.corr()
    ax = sns.heatmap(
        corr, 
        vmin=-1, vmax=1, center=0,
        cmap=sns.diverging_palette(220, 20, n=200),
        square=True
    )
    ax.set_xticklabels(
        ax.get_xticklabels(),
        rotation=45,
        horizontalalignment='right'
    );
    sns.set(rc={'figure.figsize':(15,15)})

    ################
    #Dealing with missing data
    print(len(df))

    # drop columns where weather_file is missing in the data, as it wont have the weather situation at that time, so its where ever data is 
    #missing we can remove those rows as it wont be useful
    index = df[df['weather_file'] == 'File Not Found'].index
    df.drop(index, inplace = True)
    print(len(df))


    ################
    # Weather data has a lot of 0 and values some of which may be missing values,
    # Mark '0' values in weather columns as Na (to see how many there are) 
    # As 0 wont add any value to the data, we are converting to NA and then removing them which will make data set
    subset0 = ['Temp_pre_30','Temp_pre_15','Temp_pre_7','Temp_cont','Wind_pre_30','Wind_pre_15','Wind_pre_7','Wind_cont','Hum_pre_30', 'Hum_pre_15','Hum_pre_7','Hum_cont']
    df[subset0] = df[subset0].replace({0:np.nan, '0':np.nan})
    print(len(df))

    # Mark '-1' as missing
    subset_neg1 = ['Temp_pre_30','Temp_pre_15','Temp_pre_7','Temp_cont','Wind_pre_30','Wind_pre_15','Wind_pre_7','Wind_cont','Hum_pre_30', 'Hum_pre_15','Hum_pre_7','Hum_cont','Prec_pre_30','Prec_pre_15','Prec_pre_7','Prec_cont']
    df[subset_neg1] = df[subset_neg1].replace({-1:np.nan})

    # Drop observations where all weather columns are 0
    df = df.dropna(how='all',
                        subset=['Temp_pre_30','Temp_pre_15','Temp_pre_7','Temp_cont','Wind_pre_30','Wind_pre_15','Wind_pre_7','Wind_cont',])
    print(len(df))
    # This leaves us with 38,689 observations  +/- 3,000  to work with (originally we had 50,000)

    ################
    # fill the 'pre' columns temp wind and humidity with mean values
    subset_fill_mean = ['Temp_pre_30','Temp_pre_15','Temp_pre_7', 'Wind_pre_30','Wind_pre_15','Wind_pre_7', 'Hum_pre_30', 'Hum_pre_15','Hum_pre_7']
    df[subset_fill_mean] = df[subset_fill_mean].fillna(df[subset_fill_mean].mean())

    # Fill NAs in the date of fire containment based on mean values from previous days
    for col in ['Temp','Wind','Hum']:
        df[f'{col}_cont'] = df.apply(
            lambda row: (row[f'{col}_pre_7']+row[f'{col}_pre_15']+row[f'{col}_pre_30'])/3 if np.isnan(row[f'{col}_cont']) else row[f'{col}_cont'],
            axis=1)

## Load DF 

In [4]:
from loadDFRegion import getDF
df,dfRegionList = getDF()

Grabbing region 1 dataframe...
Grabbing region 2 dataframe...
Grabbing region 3 dataframe...
Grabbing region 4 dataframe...
Grabbing region 5 dataframe...
Grabbing region 6 dataframe...
Grabbing region 8 dataframe...
Grabbing region 9 dataframe...
Grabbing region 10 dataframe...


In [5]:
#Sanity Check !!
sum1=0
ct=1
for dftemp in dfRegionList:
    if(ct>=7):
        ct+=1
    #print(ct, dftemp.size)
    sum1+= dftemp.size
print(df.size, sum1)
assert(sum1==df.size)

1445578 1445578


## Setting up Experiments

In [6]:
def experimentCreation_FireSizePrediction(df,yTarget):
    """Function return all experiments splitting data by yTarget name: 
        regression - fire_size
        classification - fire_cause
    """
    
    # Experiment 1 
    """
    - which will select all teh available  features from the dataset
    -Features included - variables related to Vegetation,Temperature, Humidity, Wind, Precipitation, cause of  fire, longitude and latitude
    - we have 34 variables  for x-variables  to which we are gonna target one y-variable which is fire_size
    - selecting features and target variables"""
    X1 = df[['Vegetation','remoteness','latitude','Temp_pre_30','Temp_pre_15','Temp_pre_7','Temp_cont','Wind_pre_30','Wind_pre_15','Wind_pre_7','Wind_cont','Hum_pre_30', 'Hum_pre_15','Hum_pre_7','Hum_cont','Prec_pre_30','Prec_pre_15','Prec_pre_7','Prec_cont','stat_cause_desc', 'longitude']]
    #X1 = df[['Vegetation_4','remoteness', 'Vegetation_9','Vegetation_12','Vegetation_14','Vegetation_15','Vegetation_16','latitude','Temp_pre_30','Temp_pre_15','Temp_pre_7','Temp_cont','Wind_pre_30','Wind_pre_15','Wind_pre_7','Wind_cont','Hum_pre_30', 'Hum_pre_15','Hum_pre_7','Hum_cont','Prec_pre_30','Prec_pre_15','Prec_pre_7','Prec_cont','Cause_Debris Burning','Cause_Equipment Use','Cause_Fireworks','Cause_Lightning','Cause_Miscellaneous','Cause_Missing/Undefined','Cause_Powerline','Cause_Railroad','Cause_Smoking','Cause_Structure', 'longitude']]
    # X1 = df[['Vegetation_4','Vegetation_9','Vegetation_12','Vegetation_14','Vegetation_15','Vegetation_16','latitude','Temp_pre_30','Temp_pre_15','Temp_pre_7','Temp_cont','Wind_pre_30','Wind_pre_15','Wind_pre_7','Wind_cont','Hum_pre_30', 'Hum_pre_15','Hum_pre_7','Hum_cont','Prec_pre_30','Prec_pre_15','Prec_pre_7','Prec_cont','Cause_Debris Burning','Cause_Equipment Use','Cause_Fireworks','Cause_Lightning','Cause_Miscellaneous','Cause_Missing/Undefined','Cause_Powerline','Cause_Railroad','Cause_Smoking','Cause_Structure', 'longitude']]

    y = df[yTarget] 

    #train test split
    X1_train, X1_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=42)
    df1 = [X1_train, X1_test, y_train, y_test]
    #######################
    #Experiment type 2 
    """-Include only long, lat, vegetation, cause and pre- weather data, without cont
    - which is the data set where I removed the variables  on which the fire is  containining on the day
    - removed 4 variables
    - selecting features and target variables"""
    X2 = df[['Vegetation','latitude','Temp_pre_30','Temp_pre_15','Temp_pre_7','Wind_pre_30','Wind_pre_15','Wind_pre_7','Hum_pre_30', 'Hum_pre_15','Hum_pre_7','Prec_pre_30','Prec_pre_15','Prec_pre_7','stat_cause_desc', 'longitude']]
    #X2 = df[['Vegetation_4','Vegetation_9','Vegetation_12','Vegetation_14','Vegetation_15','Vegetation_16','latitude','Temp_pre_30','Temp_pre_15','Temp_pre_7','Wind_pre_30','Wind_pre_15','Wind_pre_7','Hum_pre_30', 'Hum_pre_15','Hum_pre_7','Prec_pre_30','Prec_pre_15','Prec_pre_7','Cause_Debris Burning','Cause_Equipment Use','Cause_Fireworks','Cause_Lightning','Cause_Miscellaneous','Cause_Missing/Undefined','Cause_Powerline','Cause_Railroad','Cause_Smoking','Cause_Structure', 'longitude']]
    y = df[yTarget]

    #train test split
    X2_train, X2_test, y_train, y_test = train_test_split(X2, y, test_size=0.2, random_state=42)
    df2 = [X2_train, X2_test, y_train, y_test]
    ########################
    #Experiment 3 
    #- Including only lat, long and weather pre- data
    #When I have done the feature importance, I got to know that the cause and vegetation is not that important, so here we removed the 2 
    #  selecting features and target variables
    X3 = df[['latitude','longitude','Temp_pre_30','Temp_pre_15','Temp_pre_7','Wind_pre_30','Wind_pre_15','Wind_pre_7','Hum_pre_30', 'Hum_pre_15','Hum_pre_7','Prec_pre_30','Prec_pre_15','Prec_pre_7']]
    y = df[yTarget]

    #train test split
    X3_train, X3_test, y_train, y_test = train_test_split(X3, y, test_size=0.2, random_state=42)
    df3 = [X3_train, X3_test, y_train, y_test]
    
    ########################
    #Experiment 4 
    #with experiment 1 data with normalization
    # have done the minMax normalization for the experiment 1 data frame.
    df_4 = df[['Vegetation','latitude','Temp_pre_30','Temp_pre_15','Temp_pre_7','Temp_cont','Wind_pre_30','Wind_pre_15','Wind_pre_7','Wind_cont','Hum_pre_30', 'Hum_pre_15','Hum_pre_7','Hum_cont','Prec_pre_30','Prec_pre_15','Prec_pre_7','Prec_cont','stat_cause_desc', 'longitude']]
    names = df_4.columns

    # normalizing data
    df_4 = preprocessing.normalize(df_4)
    scaled_df = pd.DataFrame(df_4, columns=names)

    #train test split
    X4_train, X4_test, y_train, y_test = train_test_split(scaled_df, y, test_size=0.2, random_state=42)
    df4 = [X4_train, X4_test, y_train, y_test]
    
    return [df1,df2,df3,df4]

## Running all regression models on all experiments for Prediction on Fire Size

In [ ]:
#regionCount=1
regionExperimentDict = {}
for i in range(len(dfRegionList)):
    dfRegion = dfRegionList[i]
    regionCount =i+1
    if(regionCount>=7):
        regionCount+=1
    
    experimentList = experimentCreation_FireSizePrediction(dfRegion,"fire_size")
    print(f"\n\n____Running all experiments for Region {regionCount}____")
    print("____________________________________________")
    print("____________________________________________")
    experimentListOfDictionaries = []
    for i in range(len(experimentList)):
        if(i not in [1,2,3]):
            experiment=experimentList[i]
            print(f"\n--------------Experiment {i+1}--------------")
            print(experiment[0].size,experiment[1].size,experiment[2].size,experiment[3].size)
            regressorDict = run_all_regressors(experiment[0],experiment[2],experiment[1],experiment[3])
            experimentListOfDictionaries.append(regressorDict)
    print("____________________________________________")
    
    regionExperimentDict[regionCount]= experimentListOfDictionaries
    



____Running all experiments for Region 1____
____________________________________________
____________________________________________

--------------Experiment 1--------------
17241 4326 821 206
Running RandomForestRegressor
Score on training data: 0.980933163326186
Score on testing data: 0.8617670864622274
Mean Absolute Error:  0.03928680008848228
R Squared:  0.8617670864622274

Running GradientBoostingRegressor
Score on training data: 0.9743691746910226
Score on testing data: 0.8636906263205764
Mean Absolute Error:  0.03799259763637053
R Squared:  0.8636906263205764

Running DecisionTreeRegressor
Score on training data: 0.9999999760149556
Score on testing data: 0.8412387659327913
Mean Absolute Error:  0.034719173288736235
R Squared:  0.8412387659327913

Running ExtraTreesRegressor
Score on training data: 0.9999999760149556
Score on testing data: 0.8628135154455685
Mean Absolute Error:  0.037727878844816966
R Squared:  0.8628135154455685

Running SVR
Score on training data: 0.02366

In [ ]:
for i in range(10):
    regionCt = i+1
    if(regionCt>=7):
        regionCt+=1
    print(f"Results from Region {regionCt}")
    print(regionExperimentDict[regionCt])

In [ ]:
#regionExperimentDict

In [ ]:
def createRegionDFResults(regressorDict):
    dfRegressionResults = pd.DataFrame(columns=['ModelName','TrainScore','TestScore','MAE','R^2'])
    for key in regressorDict.keys():
        #print(key)
        resultList = regressorDict[key]
        new_row = {'ModelName':key, 'TrainScore':resultList[0], 'TestScore':resultList[1], 'MAE':resultList[2] , 'R^2':resultList[3]}
        
        dfRegressionResults = dfRegressionResults.append(new_row, ignore_index=True)
    return dfRegressionResults

In [ ]:
regionRegressionList=[]
for i in range(1,len(regionExperimentDict)+1):
    
    if(i>=7):
        i+=1
    #print(i)
    regionRegressionResults = createRegionDFResults(regionExperimentDict[i][0])
    regionRegressionList.append(regionRegressionResults)

In [ ]:
for i in range(len(regionRegressionList)):
    dfTemp = regionRegressionList[i]
    regionNum= i+1
    if(regionNum>=7):
        regionNum+=1
    print(f"\nRegion {regionNum}")
    print(dfTemp.head())

In [ ]:
#TODO select best model and put in region table


## Running all Classification Models for Fire Cause Prediction

In [ ]:
# Encode categorical values to numeric
df['stat_cause_descr'] = df['stat_cause_descr'].apply(lambda x: cause_encoded_dist[x]).astype('int')
df.head()

In [ ]:
regionCount=1
for dfRegion in dfRegionList:
    if(regionCount>=7):
        regionCount+=1
    
    experimentList = experimentCreation_FireSizePrediction(dfRegion)
    print(f"\n\n____Running all experiments for Region {regionCount}____")
    print("____________________________________________")
    print("____________________________________________")
    for i in range(len(experimentList)):
        experiment=experimentList[i]
        print(f"\n--------------Experiment {i+1}--------------")
        print(experiment[0].size,experiment[1].size,experiment[2].size,experiment[3].size)
        run_all_regressors(experiment[0],experiment[2],experiment[1],experiment[3])
    print("____________________________________________")
    regionCount+=1

In [ ]:
print(

### Hyper-parameter tuning: TODO Have not done

In [ ]:
rf_reg = RandomForestRegressor()
search_grid={'n_estimators':[50,100,200],'max_depth':[2,5,8,10]}
search=GridSearchCV(estimator=rf_reg,param_grid=search_grid,scoring='neg_mean_absolute_error',n_jobs=1,cv=5, verbose=1)
search.fit(df1[0], df1[2])
print(search.best_score_)
print(search.best_params_)


In [ ]:
# Random Forest Regressor 
rf_reg = RandomForestRegressor(n_estimators = 200, max_depth=10)

# fit the regressor with x and y data
rf_reg.fit(df1[0], df1[2])

predictions = rf_reg.predict(df1[1])
print('Mean Absolute Error:', metrics.mean_absolute_error(df1[3], predictions))
print('R Squared:', metrics.r2_score(df1[3], predictions))

### Neural network model (from other nb) will run later

In [ ]:
# selecting features and target variables
X = df[['Vegetation_4','Vegetation_9','Vegetation_12','Vegetation_14','Vegetation_15','Vegetation_16','latitude','Temp_pre_30','Temp_pre_15','Temp_pre_7','Temp_cont','Wind_pre_30','Wind_pre_15','Wind_pre_7','Wind_cont','Hum_pre_30', 'Hum_pre_15','Hum_pre_7','Hum_cont','Prec_pre_30','Prec_pre_15','Prec_pre_7','Prec_cont','Cause_Debris Burning','Cause_Equipment Use','Cause_Fireworks','Cause_Lightning','Cause_Miscellaneous','Cause_Missing/Undefined','Cause_Powerline','Cause_Railroad','Cause_Smoking','Cause_Structure', 'longitude']]
y = df['fire_size']

#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(X_train))

In [ ]:
X_train.shape

In [ ]:
model = Sequential([
    #normalizer,
    Dense(34, activation="relu", kernel_regularizer = regularizers.l2(0.01), kernel_initializer='normal',input_dim = X_train.shape[1]),
    Dense(34, activation="relu", kernel_regularizer = regularizers.l2(0.01)),
    Dense(64, activation="relu", kernel_regularizer = regularizers.l2(0.01)),
    Dense(32, activation="relu", kernel_regularizer = regularizers.l2(0.01)),
    Dense(1, activation='linear')
])

model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['accuracy','mae']
)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    validation_split=0.15,
    callbacks=[tf.keras.callbacks.EarlyStopping(
        monitor='val_mae',
        patience=20,
        mode='min',
        restore_best_weights=True
    )]
)

In [ ]:
test_loss, test_acc, test_mae = model.evaluate(x=X_test, y=y_test, verbose=0)
print('Mean Absolute Error: {acc:0.3f}'.format(acc=test_mae))
print('accuracy: {acc:0.3f}'.format(acc=test_acc))
print('loss: {acc:0.3f}'.format(acc=test_loss))

In [ ]:
print(f"Average Fire Size: {df.fire_size.mean()}")
print(f"Standard Deviation of Fire Size: {df.fire_size.std()}")


### end here